In [8]:
from google.colab import drive

drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [9]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [10]:
!pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [11]:
from huggingface_hub import notebook_login
import pandas as pd
from datasets import load_dataset, DatasetDict, load_metric
from transformers import AutoModelForQuestionAnswering, TrainingArguments, Trainer
from transformers import AutoTokenizer
from tqdm.auto import tqdm
import numpy as np
import collections
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
from datasets import load_metric

In [12]:
notebook_login()

## Load Validation Dataset

#### Directly loading the preprocessed data from training phase

### Processed data is present in the Data folder

In [14]:
validate_data_json='/content/gdrive/MyDrive/csci544-NLP/NLP-Group-Project/final/validate_json.json'
validate_dataset = load_dataset('json', data_files=validate_data_json)

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Dataset json downloaded and prepared to /root/.cache/huggingface/datasets/json/default-f624fc6245893852/0.0.0/fe5dd6ea2639a6df622901539cb550cf8797e5a6b2dd7af1cf934bed8e233e6e. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

In [15]:
valid_dict = DatasetDict({'valid': validate_dataset['train']})

## Loading the trained Model

### the link to download our trained model: https://drive.google.com/file/d/1xJK-r6Z7Zubm1o8CCkFUKan3U-Q1CVkt/view?usp=share_link

### After downloading the "deberta-trained.zip", unzip it into a folder named "trainedmodel" 

In [16]:
!unzip '/content/gdrive/MyDrive/csci544-NLP/NLP-Group-Project/final/deberta-trained.zip' -d trainedmodel

Archive:  /content/gdrive/MyDrive/csci544-NLP/NLP-Group-Project/final/deberta-trained.zip
   creating: trainedmodel/content/deberta-trained/
  inflating: trainedmodel/content/deberta-trained/tokenizer_config.json  
  inflating: trainedmodel/content/deberta-trained/training_args.bin  
  inflating: trainedmodel/content/deberta-trained/config.json  
  inflating: trainedmodel/content/deberta-trained/special_tokens_map.json  
  inflating: trainedmodel/content/deberta-trained/pytorch_model.bin  
  inflating: trainedmodel/content/deberta-trained/tokenizer.json  


### make sure model path is given correctly here



In [17]:
modelPath = "/content/trainedmodel/content/deberta-trained"
tokenizer = AutoTokenizer.from_pretrained(modelPath)

In [18]:
max_length = 384 # The maximum length of a feature (question and context)
doc_stride = 128 # The authorized overlap between two part of the context when splitting it is needed.

In [21]:
def prepare_validation_features(examples):
    # Some of the questions have lots of whitespace on the left, which is not useful and will make the
    # truncation of the context fail (the tokenized question will take a lots of space). So we remove that
    # left whitespace
    examples["question"] = [q.lstrip() for q in examples["question"]]

    # Tokenize our examples with truncation and maybe padding, but keep the overflows using a stride. This results
    # in one example possible giving several features when a context is long, each of those features having a
    # context that overlaps a bit the context of the previous feature.
    tokenized_examples = tokenizer(
        examples["question"],
        examples["context"],
        truncation="only_second",
        max_length=max_length,
        stride=doc_stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    # Since one example might give us several features if it has a long context, we need a map from a feature to
    # its corresponding example. This key gives us just that.
    sample_mapping = tokenized_examples.pop("overflow_to_sample_mapping")

    # We keep the example_id that gave us this feature and we will store the offset mappings.
    tokenized_examples["example_id"] = []

    for i in range(len(tokenized_examples["input_ids"])):
        # Grab the sequence corresponding to that example (to know what is the context and what is the question).
        sequence_ids = tokenized_examples.sequence_ids(i)
        context_index = 1

        # One example can give several spans, this is the index of the example containing this span of text.
        sample_index = sample_mapping[i]
        tokenized_examples["example_id"].append(examples["id"][sample_index])

        # Set to None the offset_mapping that are not part of the context so it's easy to determine if a token
        # position is part of the context or not.
        tokenized_examples["offset_mapping"][i] = [
            (o if sequence_ids[k] == context_index else None)
            for k, o in enumerate(tokenized_examples["offset_mapping"][i])
        ]

    return tokenized_examples

In [22]:
tokenized_valid = valid_dict.map(prepare_validation_features, batched=True, remove_columns=valid_dict["valid"].column_names)

In [23]:
tokenized_valid

DatasetDict({
    valid: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'offset_mapping', 'example_id'],
        num_rows: 735
    })
})

In [24]:
model = AutoModelForQuestionAnswering.from_pretrained(modelPath)

## Prediction

In [25]:

def postprocess_qa_predictions(examples, features, raw_predictions, n_best_size = 20, max_answer_length = 30):
    all_start_logits, all_end_logits = raw_predictions
    # Build a map example to its corresponding features.
    example_id_to_index = {k: i for i, k in enumerate(examples["id"])}
    features_per_example = collections.defaultdict(list)
    for i, feature in enumerate(features):
        features_per_example[example_id_to_index[feature["example_id"]]].append(i)

    # The dictionaries we have to fill.
    predictions = collections.OrderedDict()

    # Logging.
    print(f"Post-processing {len(examples)} example predictions split into {len(features)} features.")

    # Let's loop over all the examples!
    for example_index, example in enumerate(tqdm(examples)):
        # Those are the indices of the features associated to the current example.
        feature_indices = features_per_example[example_index]

        min_null_score = None # Only used if squad_v2 is True.
        valid_answers = []
        
        context = example["context"]
        # Looping through all the features associated to the current example.
        for feature_index in feature_indices:
            # We grab the predictions of the model for this feature.
            start_logits = all_start_logits[feature_index]
            end_logits = all_end_logits[feature_index]
            # This is what will allow us to map some the positions in our logits to span of texts in the original
            # context.
            offset_mapping = features[feature_index]["offset_mapping"]

            # Update minimum null prediction.
            cls_index = features[feature_index]["input_ids"].index(tokenizer.cls_token_id)
            feature_null_score = start_logits[cls_index] + end_logits[cls_index]
            if min_null_score is None or min_null_score < feature_null_score:
                min_null_score = feature_null_score

            # Go through all possibilities for the `n_best_size` greater start and end logits.
            start_indexes = np.argsort(start_logits)[-1 : -n_best_size - 1 : -1].tolist()
            end_indexes = np.argsort(end_logits)[-1 : -n_best_size - 1 : -1].tolist()
            for start_index in start_indexes:
                for end_index in end_indexes:
                    # Don't consider out-of-scope answers, either because the indices are out of bounds or correspond
                    # to part of the input_ids that are not in the context.
                    if (
                        start_index >= len(offset_mapping)
                        or end_index >= len(offset_mapping)
                        or offset_mapping[start_index] is None
                        or offset_mapping[end_index] is None
                    ):
                        continue
                    # Don't consider answers with a length that is either < 0 or > max_answer_length.
                    if end_index < start_index or end_index - start_index + 1 > max_answer_length:
                        continue

                    start_char = offset_mapping[start_index][0]
                    end_char = offset_mapping[end_index][1]
                    valid_answers.append(
                        {
                            "score": start_logits[start_index] + end_logits[end_index],
                            "text": context[start_char: end_char]
                        }
                    )
        
        if len(valid_answers) > 0:
            best_answer = sorted(valid_answers, key=lambda x: x["score"], reverse=True)[0]
        else:
            # In the very rare edge case we have not a single non-null prediction, we create a fake prediction to avoid
            # failure.
            best_answer = {"text": "", "score": 0.0}
        
        # Let's pick our final answer: the best one or the null answer (only for squad_v2)
        if not True:
            predictions[example["id"]] = best_answer["text"]
        else:
            answer = best_answer["text"] if best_answer["score"] > min_null_score else ""
            predictions[example["id"]] = answer

    return predictions

In [26]:
trainer = Trainer(model=model)


In [27]:
raw_predictions = trainer.predict(tokenized_valid['valid'])

In [28]:
final_predictions = postprocess_qa_predictions(valid_dict['valid'], tokenized_valid['valid'], raw_predictions.predictions)

Post-processing 335 example predictions split into 735 features.


  0%|          | 0/335 [00:00<?, ?it/s]

## Evaluation

In [29]:
predictions = [ v for k, v in final_predictions.items()]
references = [ex["answers"]["text"] for ex in valid_dict['valid']]

In [30]:

# remove empty string elements from predictions and references
indices_to_remove = [i for i, p in enumerate(predictions) if p == ""]
for i in reversed(indices_to_remove):
    del predictions[i]
    del references[i]

print(predictions)  
print(references) 

['Sprite', 'Smoky Paprika-Baked Garbanzo Beans', ' Anthony Bourdain', ' Mike Rounds', ' Jalapeno Popper Quesarito', ' Rag & Bone', ' pixie cut', ' southern flying squirrel', ' antique cabinet', ' August 6th', 'The Arizona Republic', ' Tim Masthay', ' squash', ' $25K', ' Crazy Frog', ' Dieffenbachia', ' baby food', ' Kirk Douglass', ' 1,4-dioxane', ' Wyoming', '2.1 coffee drinks per day', ' giant piece of cardboard', ' Switzerland', ' make-shift tap shoes', ' two foundations', ' Boxed', ' $700 million', ' Gilbert Arenas', ' Alex Castellanos', ' "Gravity', ' Wisconsin', ' Isla Fisher', ' never made a dollar', ' Atlanta', ' Israelis', ' Chiwetel Ejiofor', ' Lviv', ' joins in', ' June 12 at 5 PM ET', ' R. Kelly', ' $1,000 per haircut', ' Wiz Khalifa', ' Fab, right?', ' Let Hillary live!', 'VLC', ' 19 warm sweaters', ' Black Women', "I think she's fabulous", ' dying his hair sky blue', ' underarm deodorant', 'Arthur Brooks', 'Antenna Bands', ' his own clothes', " China's Sichuan Province", 

## BLEU

In [41]:
# try with different n-gram orders
bleuscore = sentence_bleu(references, predictions, weights=[.2,0,0,0])
print(f'BLEU score:',bleuscore)

BLEU score: 0.6543893899412373


## METEOR

In [36]:
meteor = load_metric("meteor")

<ipython-input-36-a91dd7fa38e0>:1: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  meteor = load_metric("meteor")


[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


In [39]:
score = meteor.compute(predictions=predictions, references=references)
alpha=.6
meteor_score = (1 - alpha) * score['meteor'] + alpha * len(predictions) / len(references)
meteor_score

0.6604630017067953